# Bi-Cluster Recommender and Clustering Algorithm

This code can work for multiple datasets. The example shown is for a dataset that lists countries and contains average, per capital alcohol consumption for beer, wine, and other spirits. The Dendogram created clusters countries based on their similarity (Pearson, Euclid, or mixed).


In [2]:
###########################################
#                                         #
#     Bi-cluster Classifier for           #
#     similarity and recommendation       #
#                                         #
#     Created By: Sam Showalter           #
#     Creation Date: 5/28/2017            #
#                                         #
#     Cluster code modeled after          #
#     Programming Collective              #
#     Intelligence                        #
#     by Toby Segaran                     #
#                                         #
###########################################

#Module imports and dependencies
import os
import pandas as pd
import math
from math import sqrt
from PIL import Image, ImageDraw

In [3]:
#################################
#
#        Data Collection
#
#################################


#Path to data set. Change this so your data is in your working directory
path = "/Users/Sam/Documents/Python/Recommender"
os.chdir(path)

#Import data. Change for your files as necessary
import_data = pd.read_csv('CountryA.csv')

#Reset index to what you want to compare. Change name for your data
import_data.set_index('country', inplace = True)

#Transposes data so that it can be more readily compared with TopMatches. Not necessary for cluster.
data = import_data.transpose()

#Extract the row_names and column names of the data
row_names = data.columns.tolist()
col_names = data.index.tolist()


#Collect and formats raw data into lists. Each row represents attributes of an item that will be clustered.
raw_data = []
for i in data:
    raw_data.append(data[i].tolist())

#Creates a dictionary of the data
data = data.to_dict()



In [4]:
########################
#
#       Classes
#
########################

#Cluster object used to compare data and make Dendogram
class bicluster():
    def __init__(self, vec,left = None, right = None, distance = 0.0, id = None):
        #Left child
        self.left = left
        #Right child
        self.right = right
        #Object vector
        self.vec = vec
        #Object id
        self.id = id
        #Object distance
        self.distance = distance




In [5]:
########################
#
#       Functions
#
########################

#Euclidian distance correlation for TopMatches. Allows you to specify data source
def sim_Euclid(prefs, v1, v2):

    #get the list of shared items
    si = {}
    for item in prefs[v1]:
        if item in prefs[v2]:
            si[item] = 1
            
    # if they have not ratings in common, return zero
    if len(si) == 0: 
        return 0
    
    #Add up the squares of all of the differences
    sum_of_squares = sum([pow(prefs[v1][item] - prefs[v2][item],2) for item in prefs[v1] if item in prefs[v2]])
    
    euclid = math.sqrt(1/(1 + (sum_of_squares))) 

    return euclid

#Pearson distance correlation for TopMatches. Allows you to specify data source
def sim_Pearson(data,p1,p2):

    # Get the list of mutually rated items
    si = {}
    for item in data[p1]:
        if item in data[p2]: 
            si[item]=1

    # Find the number of elements
    n=len(si)

    # if they are no ratings in common, return 0
    if n==0: return 0

    # Add up all the preferences
    sum1=sum([data[p1][it] for it in si])
    sum2=sum([data[p2][it] for it in si])

    # Sum up the squares
    sum1Sq=sum([pow(data[p1][it],2) for it in si])
    sum2Sq=sum([pow(data[p2][it],2) for it in si])

    # Sum up the products
    pSum=sum([data[p1][it]*data[p2][it] for it in si])

    # Calculate Pearson score fraction
    num = pSum - (sum1*sum2/n)
    den = sqrt((sum1Sq - pow(sum1,2)/n) * (sum2Sq - pow(sum2,2)/n))

    #So you do not divide by zero
    if den == 0: return 0

    #Calculate pearson correlation (r)
    pearson = num / den

    return pearson

#Euclidian distance correlation helper for the cluster algorithm. 
def euclid(v1, v2):

    #Add up the squares of all of the differences
    sum_of_squares = 0
    for i in range(len(v1)):
        sum_of_squares += pow((v1[i] - v2[i]),2)
    
    #If some of squares is zero, you want to ensure that dividing by zero does not ruin data
    if sum_of_squares == 0:
        #Hardcoded value to handle case of euclidian coefficient of zero.
        euclid = 0.000000000001
    else:
        euclid =  sum_of_squares
       
    return euclid

#Gives pearson correlation coefficient between two points (r). Used as a cluster helper
def pearson(v1,v2):

    #simple sums
    sum1 = sum(v1)
    sum2 = sum(v2)
    
    #Sum of the squares
    sum1Sq = sum([pow(v,2) for v in v1])
    sum2Sq = sum([pow(v,2) for v in v2])
    
    #Sum of the products
    pSum = sum([v1[i] * v2[i] for i in range(len(v1))])
    
    #Calculate r (Pearson Score)
    numerator = pSum - (sum1*sum2/len(v1))
    denominator = sqrt((sum1Sq - pow(sum1,2)/len(v1))*(sum2Sq - pow(sum2, 2)/len(v1)))
    if denominator == 0: return 0
    
    return 1.0 - numerator / denominator

#Gives the most similar items for each value
def topMatches(data, value, n, similarity):
    #Gets top matches for each item
    res_list = [(similarity(data,value,other),other) for other in data if other != value]
    
    #Sort the similarity results
    res_list.sort()
    
    #Put result correlations in descending order
    res_list.reverse()
    
    #Can turn list into a dataframe for easier viewing
    res_list = pd.DataFrame(res_list)
    
    return res_list[0:n]
 
#Gives an average correlation between pearson and euclidian distance    
def combinedTopMatch(data,value,n):
    #Combines the different correlation coefficients to get the best fit
    res_list = [((sim_Pearson(data,value,other)+sim_Euclid(data,value,other))/2,other) for other in data if other != value]
    
    #Sort the similarity results
    res_list.sort()
    
    #Put result correlations in descending order
    res_list.reverse()
    
    #Can turn list into a dataframe for easier viewing
    res_list = pd.DataFrame(res_list)
    
    return res_list[0:n]


#Cluster algorithm. Can alter the method of comparison from pearson correlation to euclidian or mixed
def hcluster(rows, distance = pearson):
    distances = {}
    currentclustid = -1
    
    #clusters are initially just the rows
    clust = [bicluster(rows[i],id = i) for i in range(len(rows))]
    
    while len(clust) > 1:
        lowestpair = (0,1)
        closest  = distance(clust[0].vec,clust[1].vec)
        
        #loop through every pair looking for the smallest distance
        for i in range(len(clust)):
            for j in range(i + 1, len(clust)):

                #Distances is the cache of distance calculations
                if (clust[i].id,clust[j].id) not in distances:                       
                    distances[(clust[i].id,clust[j].id)]=distance(clust[i].vec,clust[j].vec)
                    
                d = distances[(clust[i].id,clust[j].id)]
                
                if d < closest:          
                    closest = d          
                    lowestpair = (i,j)
                    
        # calculate the average of the two clusters    
        mergevec=[(clust[lowestpair[0]].vec[i] + clust[lowestpair[1]].vec[i])/2.0    
                  for i in range(len(clust[0].vec))]
                
        # create the new cluster    
        newcluster=bicluster(mergevec,left=clust[lowestpair[0]],                            
                        right=clust[lowestpair[1]],                       
                        distance=closest,id=currentclustid)
                                    
        # cluster ids that weren't in the original set are negative    
        currentclustid -= 1    
        del clust[lowestpair[1]]    
        del clust[lowestpair[0]]    
        clust.append(newcluster)
        
    return clust[0]

#Prints the first n members of a certain cluster.
def printClust(clust, labels = None, n = 0):
    #Indent to make a hierarchy layout
    for i in range(n): print(" ",)
    if clust.id < 0:
        #negative id means that this is a branch
        print("-")
    else:
        #positive id means that this is an endpoint
        if labels == None: print(clust.id)
        else: print(labels[clust.id])
        
    #now print the left and right branches
    if clust.left!=None: printClust(clust.left,labels=labels,n=n+1)  
    if clust.right!=None: printClust(clust.right,labels=labels,n=n+1)

#Gets the height of the dendogram
def getheight(clust):
    # Is this an endpoint? Then the height is just 1
    if clust.left==None and clust.right==None: return 1
    # Otherwise the height is the same of the heights of
    # each branch
    return getheight(clust.left)+getheight(clust.right)

#Gets the depth of the dendogram
def getdepth(clust):
    #The distance of an endpoint is 0.0
    if clust.left==None and clust.right==None: return 0
    # The distance of a branch is the greater of its two sides
    # plus its own distance
    return max(getdepth(clust.left),getdepth(clust.right))+clust.distance

#Draws a single node for the dendogram. Called recursively by drawdendogram
def drawnode(draw,clust,x,y,scaling,labels):

    #If this is not an endpoint
    if clust.id<0:

        #Get height of branches with children
        h1=getheight(clust.left)*20
        h2=getheight(clust.right)*20

        #Get top and bottom values
        top=y-(h1+h2)/2
        bottom=y+(h1+h2)/2

        # Line length
        ll=clust.distance*scaling

        # Vertical line from this cluster to children
        draw.line((x,top+h1/2,x,bottom-h2/2),fill=(255,0,0))

        # Horizontal line to left item
        draw.line((x,top+h1/2,x+ll,top+h1/2),fill=(255,0,0))

        # Horizontal line to right item
        draw.line((x,bottom-h2/2,x+ll,bottom-h2/2),fill=(255,0,0))

        # Call the function to draw the left and right nodes
        drawnode(draw,clust.left,x+ll,top+h1/2,scaling,labels)
        drawnode(draw,clust.right,x+ll,bottom-h2/2,scaling,labels)
    else:
        # If this is an endpoint, draw the item label
        draw.text((x + 5,y - 7),labels[clust.id],(0,0,0))


#Draws the dendogram. Calls the drawnode function recursively.
def drawdendrogram(clust,labels,pictype='clusters.jpg'):
     # height, width, and depth. Toy with these for aesthetics. 
    h=getheight(clust)*27
    w=17000
    depth=getdepth(clust)

    # width is fixed, so scale distances accordingly
    scaling=float(w-120)/depth

    # Create a new image with a white background
    img=Image.new('RGB',(w,h),(255,255,255))
    draw=ImageDraw.Draw(img)
    draw.line((0,h/2,10,h/2),fill=(255,0,0))

    # Draw the first node
    drawnode(draw,clust,10,(h/2),scaling,labels)
    img.save(pictype,'PNG')


In [7]:
#############################
#
#
#       Data Execution
#
#
#############################

#print(topMatches(data,'USA',20,sim_Pearson))
#print(topMatches(data,'Germany',20,sim_Euclid))
#print(combinedTopMatch(data,'USA',20))

#Create the cluster
clust=hcluster(raw_data,distance = euclid)

#Print the cluster
#printClust(clust, labels = row_names)

#Draw the dendogram of the cluster
drawdendrogram(clust,row_names,pictype='countryclust.png')

print("Finished compiling.")



Finished compiling.
